
# STEP 1: Import ALL the things!


### The packages

In [14]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=b99dbb80eedb7316d13d31df07c66ac1021233c76b6b2b641c2511e1b0f8ea8d
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [15]:
import matplotlib
matplotlib.use("Agg")
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

time: 9.97 ms


### The data

In [16]:
## mount your Google Drive folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.67 ms


In [17]:
## change into the directory where your data is 
os.chdir("drive/My Drive/data")

FileNotFoundError: ignored

time: 1.86 s


# STEP 2: Process the data

1. Get image paths. Shuffle them.
2. Loop over the paths and and load the images

Inside the loop (for each image)

* resize the image
* flatten the image
* store the (newly flattened & resized) image in 'data' array
* store the label in 'labels' array

In [19]:
print("[INFO] loading images...")
data = []
labels = []

# imagePaths = sorted(list(paths.list_images('animals')))
imagePaths = sorted(list(paths.list_images('jcrew')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image)
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

[INFO] loading images...
time: 1min 43s


In [20]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

time: 10.7 ms


In [21]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

time: 11.5 ms


## Convert the labels from integers to vectors 

NOTE: for 2-class, binary classification you should use Keras' to_categorical function instead as the scikit-learn's LabelBinarizer will not return a
vector

In [22]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

time: 7.41 ms


# STEP 3: MODEL

In [23]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

time: 58 ms


1. Initialize learning rate and # of epochs
2. Compile the model using:

* OPTIMIZER: SGD
* LOSS: categorical cross-entropy loss

NOTE: use binary_crossentropy for 2-class classification

In [24]:
INIT_LR = 0.01
EPOCHS = 75

print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...
time: 52.8 ms


# STEP 4: Train the neural network

In [25]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=32)

Train on 341 samples, validate on 114 samples
Epoch 1/75
341/341 [==============================] - 1s 2ms/step - loss: 1.1165 - acc: 0.3695 - val_loss: 1.1075 - val_acc: 0.3596
Epoch 2/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0861 - acc: 0.4252 - val_loss: 1.0684 - val_acc: 0.3509
Epoch 3/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0697 - acc: 0.4370 - val_loss: 1.1091 - val_acc: 0.3596
Epoch 4/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0495 - acc: 0.4457 - val_loss: 1.0706 - val_acc: 0.4211
Epoch 5/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0429 - acc: 0.4692 - val_loss: 1.0699 - val_acc: 0.3596
Epoch 6/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0180 - acc: 0.4663 - val_loss: 1.0424 - val_acc: 0.5088
Epoch 7/75
341/341 [==============================] - 0s 1ms/step - loss: 1.0139 - acc: 0.5718 - val_loss: 1.0202 - val_acc: 0.4474
Epoch 8/75
341/341 [==========

# STEP 5: Evaluate the trained network

In [26]:

print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('results.png')

[INFO] evaluating network...
              precision    recall  f1-score   support

       pants       0.90      0.82      0.86        33
 shirts_tops       0.86      0.95      0.90        40
       shoes       0.97      0.95      0.96        41

    accuracy                           0.91       114
   macro avg       0.91      0.91      0.91       114
weighted avg       0.91      0.91      0.91       114

time: 242 ms


# STEP 6: Test the model on new data

In [27]:
image = cv2.imread('images/jc_test.jpeg')
output = image.copy()

## doing the same pre-processing we did above
image = cv2.resize(image, (32, 32))
image = image.astype("float") / 255.0
image = image.flatten()
image = image.reshape((1, image.shape[0]))

time: 142 ms


In [34]:
preds = model.predict(image)

## take the label with the highest probability
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]
print(label, preds[0][i])

shirts_tops 0.62997866
time: 13.5 ms
